# Language Identification Model


In [21]:
### TO DEBUGGIN'
_DEBUG = True

## Load Package

In [2]:
# to load and manipulate data
import pandas as pd
import numpy as np
import os
import json
from sklearn.model_selection import train_test_split

# to make neural netowrk model, sequencing and tokenizing
import tensorflow as tf

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras import layers

# to evaluate and display results
from sklearn.metrics import  confusion_matrix, accuracy_score
import seaborn as sns
import matplotlib.pyplot as plt

## Load Data

In [3]:
## print paths
for root, dirs, files in os.walk('./data'):
    for filename in files:
        print(os.path.join(root,filename))

./data/language.json
./data/sentences.csv


In [4]:
%%time
## vamos a medir el TIME de esta shit

df = pd.read_csv("./data/sentences.csv")

CPU times: user 5.34 s, sys: 297 ms, total: 5.64 s
Wall time: 5.64 s


## Data Cleansing

In [5]:
## eliminar columna maldita que esta de mas ID
df.drop(columns=['id'],inplace=True)

In [6]:
df.head()

,lan_code,sentence
0,cmn,我們試試看！
1,cmn,我该去睡觉了。
2,cmn,你在干什麼啊？
3,cmn,這是什麼啊？
4,cmn,今天是６月１８号，也是Muiriel的生日！


In [7]:
## [x] no hay sentencias sin code - data maravillosa!!! :D!
if _DEBUG:
    display(df.describe())

In [8]:
if _DEBUG:
    ## exploracion de frequence
    df_exploration = df.groupby("lan_code").count().sort_values("sentence",ascending = False)
    #df_exploration.plot()
    
    #!# posiblemente cuando se entrene el modelo los de baja frecuencia no va a predecir correctamente
    #!# como posible opcion 1 analizar con todo y luego ver el impacto de remover esos code lan con freq 1
    
    display(df_exploration)
    count_freq_1 = df_exploration[df_exploration.sentence<=10].sum()
    print("Code LANGUAGE with less frequency than 10: {}".format(count_freq_1))
    
    del df_exploration

## Preprocessing Data - FULL dataset sentences

In [9]:
# ENCODING CLASSES (cod language)
classes_list = list(df.lan_code.unique())
print("Count of Clases: {}".format(len(classes_list)))

Count of Clases: 404


In [10]:
## crearemos dos dic para usos posteriores de transformacion

cls_to_num = {
    cls : i
    for i,cls in enumerate(classes_list)
}

num_to_cls = {
    i : cls
    for cls,i in cls_to_num.items()
}

In [11]:
## cambiar el string-code por int-code para ingresarlos al modelo

display(df.head())
df.lan_code = df.lan_code.map(cls_to_num).astype(int)
display(df.head())

,lan_code,sentence
0,cmn,我們試試看！
1,cmn,我该去睡觉了。
2,cmn,你在干什麼啊？
3,cmn,這是什麼啊？
4,cmn,今天是６月１８号，也是Muiriel的生日！


,lan_code,sentence
0,0,我們試試看！
1,0,我该去睡觉了。
2,0,你在干什麼啊？
3,0,這是什麼啊？
4,0,今天是６月１８号，也是Muiriel的生日！


## Subset Train & Test

In [12]:
# Lista de Sentencias
X = df.pop('sentence').values

# Lista de CODE LAN 
y = df.copy().values.T[0]

if _DEBUG:
    display(X)
    display(y)

In [13]:
## liberamos memoria virtual (RAM)
del df

In [14]:
X_train, X_test, y_train, y_test = train_test_split(
    X,y,
    train_size=0.8,
    random_state=50,
    shuffle = True
)

## liberamos memoria virtual (RAM)
del X, y

# Tokenization

Usaremos este metodo para procesar los datos input para el modelo:

mas info: https://towardsdatascience.com/tokenization-for-natural-language-processing-a179a891bad4


### Sentencias -> Secuencias de palabras


In [16]:
## parametros

NUM_WORDS = 1000000
MAX_LEN = 140
NUM_CLASSES = 404 # esto lo sacamos de arriba

In [17]:
def tokenize_and_sequence(
                            train_sentences,
                            test_sentences
                            ):
    tok = Tokenizer(num_words = NUM_WORDS, oov_token='<OOV>')
    tok.fit_on_texts(train_sentences)
    
    # training set
    train_seq = tok.texts_to_sequences(train_sentences)
    train_seq = pad_sequences(
                    train_seq,
                    padding = 'post',
                    maxlen = MAX_LEN,
                    truncating = 'post'
                )
    
    # testing set
    test_seq = tok.texts_to_sequences(test_sentences)
    test_seq = pad_sequences(
                    test_seq,
                    padding = 'post',
                    maxlen = MAX_LEN,
                    truncating = 'post'
                )
    if _DEBUG:
        print(tok.get_config())
    
    return train_seq, test_seq, tok

In [18]:
%%time
## call function 
X_train, X_test, tok = tokenize_and_sequence(X_train, X_test)

# WARNING: problemas de tiempo de respuesta, se debe cambiar la config del notebook

CPU times: user 2min 17s, sys: 1.48 s, total: 2min 19s
Wall time: 2min 19s


In [23]:
if _DEBUG:
    print(X_train.shape)
    print(X_test.shape)

(8273449, 140)
(2068363, 140)


# Model NLP

## Build Model

## Train Model

## Evaluate Model